In [9]:
import requests 
import json 
import requests_cache
import time
import pandas as pd 
from IPython.core.display import clear_output


/var/folders/d1/jlnbs56x4ygg0q19v619169w0000gn/T/ipykernel_53055/821500642.py:6: DeprecationWarning: Importing clear_output from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import clear_output


In [32]:
def lastfm_get(playload):
    headers = {'user-agent': 'leolikedata'}
    url = 'https://ws.audioscrobbler.com/2.0/'

    playload['api_key'] = '01c65ba6504623c5e48586febced5463'
    playload['format'] = 'json'

    response = requests.get(url, headers= headers, params= playload)
    return response 

def run_fm_etl():
    responses = []

    page = 1
    total_pages = 2 # this is just a dummy number so the loop starts

    while page <= total_pages:
        payload = {
            'method': 'chart.gettopartists',
            'limit': 500,
            'page': page
        }

        # print some output so we can see the status
        print("Requesting page {}/{}".format(page, total_pages))
        # clear the output to make things neater
        clear_output(wait = True)

        # make the API call
        response = lastfm_get(payload)

        # if we get an error, print the response and halt the loop
        if response.status_code != 200:
            print(response.text)
            break

        # extract pagination info
        page = int(response.json()['artists']['@attr']['page'])
        # for test only get 2 pages
        total_pages = 2
        # if we wish to get full pages, using following code:
        # total_pages = int(response.json()['artists']['@attr']['totalPages'])


        # append response
        responses.append(response)

        # if it's not a cached result, sleep
        if not getattr(response, 'from_cache', False):
            time.sleep(0.25)

        # increment the page number
        page += 1

        frames = [pd.DataFrame(r.json()["artists"]['artist']) for r in responses ]
        artists = pd.concat(frames)
        artists = artists.drop(['url', 'mbid', 'streamable', 'image'], axis = 1)
        artists.to_csv('artists.csv', index= False)

In [12]:
responses = []

page = 1
total_pages = 2 # this is just a dummy number so the loop starts

while page <= total_pages:
    payload = {
        'method': 'chart.gettopartists',
        'limit': 500,
        'page': page
    }

    # print some output so we can see the status
    print("Requesting page {}/{}".format(page, total_pages))
    # clear the output to make things neater
    clear_output(wait = True)

    # make the API call
    response = lastfm_get(payload)

    # if we get an error, print the response and halt the loop
    if response.status_code != 200:
        print(response.text)
        break

    # extract pagination info
    page = int(response.json()['artists']['@attr']['page'])
    # for test only get 2 pages
    total_pages = 2
    # if we wish to get full pages, using following code:
    # total_pages = int(response.json()['artists']['@attr']['totalPages'])


    # append response
    responses.append(response)

    # if it's not a cached result, sleep
    if not getattr(response, 'from_cache', False):
        time.sleep(0.25)

    # increment the page number
    page += 1

Requesting page 2/2


In [23]:
r = [r.json()['artists']['artist'] for r in responses ]

jprint(r)

[
    [
        {
            "image": [
                {
                    "#text": "https://lastfm.freetls.fastly.net/i/u/34s/2a96cbd8b46e442fc41c2b86b821562f.png",
                    "size": "small"
                },
                {
                    "#text": "https://lastfm.freetls.fastly.net/i/u/64s/2a96cbd8b46e442fc41c2b86b821562f.png",
                    "size": "medium"
                },
                {
                    "#text": "https://lastfm.freetls.fastly.net/i/u/174s/2a96cbd8b46e442fc41c2b86b821562f.png",
                    "size": "large"
                },
                {
                    "#text": "https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png",
                    "size": "extralarge"
                },
                {
                    "#text": "https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png",
                    "size": "mega"
                }
            ],
           

In [33]:
run_fm_etl()


Requesting page 2/2
